In [ ]:
# the usual
import numpy as np
import matplotlib.pyplot as plt
import scipy.special

%matplotlib notebook

# course utils for 3D plots
import pboc_utils as pboc

In this script, we will look at the distribution of polymer lengths under
some different types of regulation. We will first think of some cytoskeletal
polymer growing in an unlimited pool of available monomers (tubulin dimers)
and the case in which the shortening of the polymer is dependent on the total
length.

#### Length - independent decay rate
At first we'll assume that the production
rate and the depoymerization rate are independent of length. Our master
equation becomes
\begin{align}
 P(l, t + dt) = P(l, t) + (rdt) P(l-1, t) + (\gamma dt) P(l+1, t) 
                 - (r dt) P(l, t) - (\gamma dt) P(l, t)
\end{align}

where $l$ is the length of the polymer, $t$ is the time, $dt$ is the timestep, $r$
is the growth rate of the polymer, and $\gamma$ is the depolymerization rate.

To begin, we'll define and initialize a set of parameters and run "spread the butter" integration as we have before.

In [ ]:
r = 5  # addition rate in monomers per time
gamma = 6  # Degradation rate in monomers per time
dt = 0.01 # Time step for butter spreading.
tot_length = 30  # Maximum polymer length
tot_time = 2000   # Time to integrate in time steps

# Set up the two-dimensional matrix of probabilities.
prob = np.zeros((tot_length + 1, tot_time))

# Set our initial condition.
prob[0, 0] = 1.0

# Start the integration over time
for t in range(1, tot_time):
    # handle the boundaries, first l=0
    prob[0, t] = prob[0, t-1] - r*dt*prob[0, t-1] + gamma*dt*prob[1, t-1]
    # then l=tot_length
    prob[-1, t] = prob[-1, t-1] + r*dt*prob[-2, t-1] - gamma*dt*prob[-1, t-1]
    # loop over polymer lengths
    for ell in range(1, tot_length - 1):
        # update master equation step.
        prob[ell, t] = prob[ell, t-1] + gamma*dt*prob[ell+1, t-1] \
                            + r*dt*prob[ell-1, t-1] - r*dt*prob[ell, t-1] \
                                        - gamma*dt*prob[ell, t-1]

In [ ]:
# Now we'll show the plot in three dimensions.
pboc.bar3(prob, xlabel='time (steps)', ylabel='polymer length in monomers',
          zlabel='P(l, t)', bin_step=int(tot_time/30))

In [ ]:
# 2D plot of a few time slices
n_slices = 5
slice_int = int(tot_time/n_slices)
len_array = np.arange(tot_length+1)
# mean length
mean_ell = r/gamma

# set up plots
_, ax = plt.subplots(figsize=(7,7))
ax.set_xlabel('polymer length')
ax.set_ylabel('probability of polymer length')
# steady state theory prediction
ax.plot(len_array, mean_ell**len_array * (1 - mean_ell), '.')
# loop over time slices and plot
for t in range(slice_int, tot_time, slice_int):
    _ = ax.plot(len_array, prob[:,t], label='time = '+str(t))

#### Length - dependent degredation rate
Now, let's look at the case in which gamma is dependent on the length of the
polymer. In essence, this means that we now can redfine gamma as some
constant times the length of the polymer.

In [ ]:
r = 0.7  # addition rate in monomers per time
gamma = 0.3  # Degradation rate in monomers per time

# Set up the two-dimensional matrix of probabilities.
prob = np.zeros((tot_length + 1, tot_time))

# Set our initial condition. nonzero just for fun.
prob[20, 0] = 1.0

# Start the integration over time
for t in range(1, tot_time):
    # handle the boundaries, first l=0
    prob[0, t] = prob[0, t-1] - r*dt*prob[0, t-1] + gamma*dt*prob[1, t-1]
    # then l=tot_length
    prob[-1, t] = prob[-1, t-1] + r*dt*prob[-2, t-1] \
                    - gamma*(tot_length)*dt*prob[-1, t-1]
    # loop over polymer lengths
    for ell in range(1, tot_length - 1):
        # update master equation step.
        prob[ell, t] = prob[ell, t-1] + gamma*(ell+1)*dt*prob[ell+1, t-1] \
                            + r*dt*prob[ell-1, t-1] - r*dt*prob[ell, t-1] \
                                        - gamma*(ell)*dt*prob[ell, t-1]

In [ ]:
# Now we'll show the plot in three dimensions.
pboc.bar3(prob, xlabel='time (steps)', ylabel='polymer length in monomers',
          zlabel='P(l, t)', bin_step=int(tot_time/30))

Let's again do a 2D plot of some time slices, including the analytical steady-state distribution (Poisson, as derived in lecture).

In [ ]:
# 2D plot, with steady-state prediction
# first compute Poisson prediction
predicted_dist = (r / gamma)**len_array * np.exp(-r/gamma) \
                   * (1 / scipy.special.gamma(len_array+1))
# set up plotting environment
_, ax = plt.subplots(figsize=(6,6))
# plot analytical steady-state distribution
ax.plot(len_array, predicted_dist, '.', label='Poisson')
# loop over time slices and plot
for t in range(slice_int, tot_time, slice_int):
    _ = ax.plot(len_array, prob[:,t], label='time = '+str(t))
ax.legend()